In [44]:
import rdflib
from rdflib import Namespace, RDF,URIRef, Literal
from owlready2 import get_ontology

In [57]:
# Define namespace
ns = Namespace("http://www.semanticweb.org/dell/ontologies/2024/3/untitled-ontology-4#")

# Load RDF file and setup RDF graph
g = rdflib.Graph()
try:
    g.parse("Movies.ttl", format="turtle")
    print("File loaded successfully")
except FileNotFoundError:
    print("File not found")

File loaded successfully


## Q1

Displays all the Persons (without a query, using inference).

In [58]:
def display_persons(owl_file):
    onto = get_ontology(owl_file).load()
    person_class = onto.search_one(iri="*Person")
    if person_class:
        for person in person_class.instances():
            print((person.name))

display_persons("data/Movies.owl")


Abdel_Rahim_Kamal
Abdelrahman_Yasser
Adel_Emam
Ahmed_Dawood
Ahmed_El_Sakka
Ahmed_Helmy
Ahmed_Malek
Ahmed_Mourad
Aimi_Samir_Ghanem
Akram_Hosny
Alaa_Al-Aswany
Amina_Khalil
Amr_Youssef
Asser_Yassin
Ayman_Bahgat_Kamar
Edward
Fares_Bakr
Hadi_El_Bagoury
Haitham_Dabbour
Hany_Adel
Hind_Sabri
Karim_Abdel_Aziz
Khaled_Diab
Khaled_El-Sawi
Khaled_Marei
Mahmoud_Hemida
Mahmoud_Yassine
Marwan_Hamed
Menna_Shalaby
Mohamad_Ramdan
Mohamed_Diab
Mohamed_Gamal_El-Adl
Mohamed_Sadeq
Nilli_Karim
Nour_El-Sherif
Shereen_Reda
Sherif_Arafa
Tamer_Ashry
Tamer_Hosny
Tara_Emad
Wael_Hamdy
Wahid_Hamed
Youssra


## Q2
Displays all the Persons (using a query, without inference). Create the used query in text file under the data folder

In [78]:
import rdflib
from rdflib import Namespace, RDF

# Define the ontology and namespace
ontology_url = "http://www.semanticweb.org/dell/ontologies/2024/3/untitled-ontology-4#"
ns = Namespace(ontology_url)

# Initialize the graph
g = rdflib.Graph()
g.namespace_manager.bind('ns', ns, override=False)

# Load the RDF data in Turtle format
try:
    g.parse("data/Movies.ttl", format="turtle")
    print("RDF data loaded successfully.")
except Exception as e:
    print(f"Failed to load RDF data: {e}")

# Read the SPARQL query from the file
try:
    with open("data/query_person_roles.sparql", "r") as file:
        query = file.read()
    print("Query loaded successfully.")
except Exception as e:
    print(f"Failed to load query: {e}")

# Execute the query and print results
try:
    qres = g.query(query)
    for row in qres:
        person_uri = row.person
        person_name = person_uri.split('#')[-1] if '#' in person_uri else person_uri
        print(f"{row.role}: {person_name}")
    if not qres:
        print("No persons found.")
except Exception as e:
    print(f"Failed to execute query: {e}")


RDF data loaded successfully.
Query loaded successfully.
Actor: Adel_Emam
Actor: Ahmed_Dawood
Actor: Ahmed_El_Sakka
Actor: Ahmed_Helmy
Actor: Ahmed_Malek
Actor: Aimi_Samir_Ghanem
Actor: Akram_Hosny
Actor: Amina_Khalil
Actor: Amr_Youssef
Actor: Asser_Yassin
Actor: Edward
Actor: Fares_Bakr
Actor: Hany_Adel
Actor: Hind_Sabri
Actor: Karim_Abdel_Aziz
Actor: Khaled_El-Sawi
Actor: Mahmoud_Hemida
Actor: Mahmoud_Yassine
Actor: Menna_Shalaby
Actor: Mohamad_Ramdan
Actor: Nilli_Karim
Actor: Nour_El-Sherif
Actor: Shereen_Reda
Actor: Tamer_Hosny
Actor: Tara_Emad
Actor: Youssra
Writer: Abdel_Rahim_Kamal
Writer: Abdelrahman_Yasser
Writer: Ahmed_Mourad
Writer: Alaa_Al-Aswany
Writer: Ayman_Bahgat_Kamar
Writer: Haitham_Dabbour
Writer: Khaled_Diab
Writer: Mohamed_Diab
Writer: Mohamed_Sadeq
Writer: Sherif_Arafa
Writer: Tamer_Hosny
Writer: Wael_Hamdy
Writer: Wahid_Hamed
Director: Hadi_El_Bagoury
Director: Khaled_Marei
Director: Marwan_Hamed
Director: Mohamed_Diab
Director: Mohamed_Gamal_El-Adl
Director: She

## Q3
displays all the Actors (without using queries, using inference).

In [64]:
# Let's create a set to avoid duplicate actors
actors = set()

# Check for all triples that involve acting in a movie
for actor, _, movie in g.triples((None, ns.isActorOf, None)):
    # Infer that this subject is an Actor
    g.add((actor, RDF.type, ns.Actor))
    actors.add(actor)

# Display all inferred actors
for actor in actors:
    actor_name = actor.split('#')[-1] if '#' in actor else actor
    print(f"Inferred Actor: {actor_name}")


Inferred Actor: Nour_El-Sherif
Inferred Actor: Fares_Bakr
Inferred Actor: Nilli_Karim
Inferred Actor: Akram_Hosny
Inferred Actor: Amina_Khalil
Inferred Actor: Karim_Abdel_Aziz
Inferred Actor: Ahmed_El_Sakka
Inferred Actor: Tamer_Hosny
Inferred Actor: Aimi_Samir_Ghanem
Inferred Actor: Tara_Emad
Inferred Actor: Hind_Sabri
Inferred Actor: Ahmed_Dawood
Inferred Actor: Ahmed_Helmy
Inferred Actor: Mahmoud_Yassine
Inferred Actor: Asser_Yassin
Inferred Actor: Menna_Shalaby
Inferred Actor: Khaled_El-Sawi
Inferred Actor: Adel_Emam
Inferred Actor: Amr_Youssef
Inferred Actor: Ahmed_Malek
Inferred Actor: Shereen_Reda
Inferred Actor: Mohamad_Ramdan
Inferred Actor: Youssra
Inferred Actor: Edward
Inferred Actor: Mahmoud_Hemida
Inferred Actor: Hany_Adel


## Q4
Reads a name of a movie


If it doesn’t exist displays an error message


Else, display its year, country, genres and actors

In [79]:
def display_movie_details(owl_file, movie_name):
    onto = get_ontology(owl_file).load()
    movie_instance = onto.search_one(iri=f"*{movie_name}")
    if movie_instance:
        print(f"Year: {movie_instance.hasYear.first()}")
        print(f"Country: {movie_instance.hasCountry.first()}")
        print(f"Genres: {[genre.name for genre in movie_instance.hasGenre]}")
        print(f"Actors: {[actor.name for actor in movie_instance.hasActor]}")
    else:
        print("Error: Movie not found.")

# Usage
display_movie_details("data/Movies.owl", "Turab_el-Mas")

Year: 2018
Country: Egypt
Genres: ['Drama']
Actors: ['Asser_Yassin', 'Tara_Emad']


## Q5

ActorWriter Relation

In [47]:
# Function to apply a specific rule from a file
def apply_rules_from_file(graph, rule_file):
    with open(rule_file, "r") as file:
        rules = file.readlines()
        
    for rule in rules:
        # Simple parsing of rule assuming the format given in previous messages
        if "ActorWriter" in rule:
            for s, p, o in graph.triples((None, ns['isActorOf'], None)):
                if (s, ns['isWriterOf'], o) in graph:
                    graph.add((s, RDF.type, ns['ActorWriter']))

# Apply rules from the rules file
apply_rules_from_file(g, "data/actor_writer.rules")

# Query to find all ActorWriters
qres = g.query(
    """
    PREFIX ns: <http://www.semanticweb.org/dell/ontologies/2024/3/untitled-ontology-4#>
    SELECT DISTINCT ?person ?name
    WHERE {
        ?person rdf:type ns:ActorWriter ;
                ns:hasName ?name .
    }
    """)

# Print results
for row in qres:
    print(f"{row.name} is both an actor and a writer.")


Tamer Hosny is both an actor and a writer.


## Q6
hasMovieCount


isOldActor


isYoungActor 

In [48]:
def apply_rules_from_file(graph, rule_file):
    with open(rule_file, "r") as file:
        rules = file.readlines()

    for rule in rules:
        rule = rule.strip()
        if "Movie Count" in rule:  # Adjusted rule name for clarity
            for actor in graph.subjects(RDF.type, ns.Actor):
                movie_count = 0
                for _, movie in graph.subject_objects(predicate=ns.isActorOf):
                    if actor == _:  # Check if the current actor is the subject
                        movie_count += 1
                graph.add((actor, ns.hasMovieCount, Literal(movie_count)))  # Added count property
        elif "Old Actors" in rule:
            age_limit = int(rule.split(", Age > ")[1])
            for actor in graph.subjects(RDF.type, ns.Actor):
                age = graph.value(actor, ns.hasAge)
                if age and int(age) > age_limit:
                    graph.add((actor, ns.isOldActor, Literal(True)))
        elif "Young Actors" in rule:
            age_limit = int(rule.split(", Age < ")[1])
            for actor in graph.subjects(RDF.type, ns.Actor):
                age = graph.value(actor, ns.hasAge)
                if age and int(age) < age_limit:
                    graph.add((actor, ns.isYoungActor, Literal(True)))

# Apply rules from the rules file
apply_rules_from_file(g, "data/actor_rules.rules")

# Query and print results
qres = g.query(
    """
    PREFIX ns: <http://www.semanticweb.org/dell/ontologies/2024/3/untitled-ontology-4#>
    SELECT ?actorName ?movieCount ?isOld ?isYoung
    WHERE {
        ?actor rdf:type ns:Actor .
        ?actor ns:hasName ?actorName .
        OPTIONAL { ?actor ns:hasMovieCount ?movieCount . }
        OPTIONAL { ?actor ns:isOldActor ?isOld . }
        OPTIONAL { ?actor ns:isYoungActor ?isYoung . }
    }
    """)

# Print results
for row in qres:
    print(f"actorName: {row.actorName}, Movie Count: {row.movieCount}, Is Old: {row.isOld}, Is Young: {row.isYoung}")


actorName: Adel Emam, Movie Count: 1, Is Old: true, Is Young: None
actorName: Ahmed Dawood, Movie Count: 1, Is Old: None, Is Young: None
actorName: Ahmed El Sakka, Movie Count: 1, Is Old: None, Is Young: None
actorName: Ahmed Helmy, Movie Count: 2, Is Old: None, Is Young: None
actorName: Ahmed Malek, Movie Count: 1, Is Old: None, Is Young: true
actorName: Aimi Samir Ghanem, Movie Count: 1, Is Old: None, Is Young: None
actorName: Akram Hosny, Movie Count: 1, Is Old: None, Is Young: None
actorName: Amina Khalil, Movie Count: 2, Is Old: None, Is Young: true
actorName: Amr Youssef, Movie Count: 1, Is Old: None, Is Young: None
actorName: Aser Yassin, Movie Count: 1, Is Old: None, Is Young: None
actorName: Edward, Movie Count: 1, Is Old: None, Is Young: None
actorName: Fares Bakr, Movie Count: 1, Is Old: true, Is Young: None
actorName: Hany Adel, Movie Count: 1, Is Old: None, Is Young: None
actorName: Hind Sabri, Movie Count: 2, Is Old: None, Is Young: None
actorName: Karim Abdel Aziz, Movie